# Trading Mean Reversion

Before even thinking of trading a mean reverting signal, read [Why I won't teach pair trading to my students](https://www.marketwatch.com/story/why-i-wont-teach-pair-trading-to-my-students-2012-10-01). 

Never forget the fundamental drivers which should drive the mean reversion, and scenarios when the drivers can dissapear.



## The trigger strategies

As the name implies, a 'trigger' represents a level of the mean reverting signal that is used by the trader to enter and exit a trade.  

### Standard Deviation multiples

A common tactic ([see quantopian](https://www.quantopian.com/posts/trading-strategy-mean-reversion)) is to check how far away from the mean the current value is, and be long (or short) if the trade is more than a predefined multiple of standard deviations (one, two or more) and setting up a stop loss at another multiple. The academic [literature](https://arxiv.org/ftp/arxiv/papers/1602/1602.05858.pdf) usually verifies the behaviour of this trading strategy out-of-sample, but the decision tends to be very binary.

Using the 'pca' signal from our past [notebook](https://drive.google.com/file/d/1QNtBrFLsn7sqayEKgJVXg0UJz1JNYgVI/view?usp=sharing), we have computed the following parameters:

In [1]:
# Request: allow HTTP-post API calls
import requests
# Json: to format as json calls to API
import json 
# Pandas: to plot the timeseries
import pandas as pd
# Datetime: to convert different date formats
import datetime as dt
# Import the Time Series library
import statsmodels.tsa.stattools as ts

from numpy import cumsum, log, polyfit, sqrt, std, subtract, sign, exp, max

/usr/local/lib/python2.7/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:

# The numbers below were obtained from a previous notebook,
# I hardwire them instead of linking the previous data
currentValue = -0.11513815488258894
mu           = -0.26912068157010066
theta        =  2.826060587147701
sigma        =  0.2638419146133024

# derived from the numbers above
half_life    = 89.58477707480525
std_band     =  0.11097829743754704

# Number of standard deviations from the mean
print ("Std's from mean:",(currentValue - mu)/std_band)


("Std's from mean:", 1.3875012524332992)


At the time of writing the pca signal was about 1.39 std's from away from the mean - so somewhere in the start of a trigger (although the most preferred multiple seems to be 2, so most likely we would be sitting out this trade if we were followers of this rule)

### Ex-ante Sharpe Ratio
However, there is no reason to limit the trigger to be standard deviation multiples. We could instead compute the ex-ante Sharpe Ratio using a known holding period, and set up a threshold in terms of Sharpe Ratio (which is more intuitive for some financial professionals) - quoting from [InvestoPedia](https://www.investopedia.com/ask/answers/010815/what-good-sharpe-ratio.asp):


> Usually, any Sharpe ratio greater than 1 is considered acceptable to good by investors. A ratio higher than 2 is rated as very good, and a ratio of 3 or higher is considered excellent. The basic purpose of the Sharpe ratio is to allow an investor to analyze how much greater a return he or she is obtaining in relation to the level of additional risk taken to generate that return.

notes 
* the number computed below is actually the Expected Return divided by the Standard Deviation of the simulation, we need to substract the risk free rate divided by the Standard Deviation, and 
* it assumes the trader will **hold** the trade from entry and will only exit at the horizon date (regadless of the path the tarde takes in between). This assumption is made by practitioners because there is an easy formula to use, **but** does not reflect reality (more below).





In [3]:
# Using the formulas to compute the forward expected mean and standard devition at some time
# t in the future (today being time '0', and expressed in terms of a year being one unit)
def OUmu_t(currentValue, mu, theta, t):
  return mu + (currentValue - mu)*exp(-theta * t)

def OUsigma_t(sigma, theta, t):
  return sqrt (  ( 1 - exp(-2 * theta * t)) * sigma * sigma / (2 * theta) )

# Set up the forward horizon
# Above, the pca halflife was 89.58, so we take roughly
# 2 x halflife = 180
NumberFP = 180
forward_period = NumberFP / 365.25

# Let's compute what will be the expected mean at NumberFP calendar days,
ExpectedMu = OUmu_t(currentValue, mu, theta, forward_period)
# and the expected standard deviation as well
ExpectedSigma = OUsigma_t(sigma, theta, forward_period)

print ("Mean after ",NumberFP," days:",ExpectedMu)
print ("Std after ",NumberFP," days:",ExpectedSigma)

# Sharpe (assuming risk free rate is 0):
AnnualisedSharpe = (currentValue - ExpectedMu)/ ExpectedSigma * sqrt(1 / forward_period)

print ('Sharpe (annual)',AnnualisedSharpe) 

('Mean after ', 180, ' days:', -0.23087160812325994)
('Std after ', 180, ' days:', 0.1074999995836604)
('Sharpe (annual)', 1.533590144519143)


The ex-ante annualised Sharpe Ratio of 1.5 is a 'good' one (as defined by Investopedia), but as practitioners assume a degradation of the ex-post SharpeRatio they tend to increase the threshold accordingly - the trading strategy would require a Sharpe roughly 50% higher than what the practitioner wants to achieve. Hence, using this trigger method would have us sitting out of the trade.

However, this formula has some unpleasant sideffects: it strongly assumes the trader will buy and hold the position to the horizon, regardless if it passes through the mean way before the end of the horizon ! (in real life the trader would have taken profits as it get closer to the mean).

### Ex ante 'real life' Sharpe Ratio (include profit target, stop loss)

Getting analytical formulas Sharpe Ratios (or other statistics like 'hitting times') of mean reverting stochastic processes is quite [complicated](https://pdfs.semanticscholar.org/755b/1db0077f87103aef38ab5e1167f1745b6140.pdf), but the magic of Moore's law allows us to run MonteCarlo simulators (provided by our Markov API) to compute the parameters (in the past MonteCarlo was usually dismissed as too compute-intensive). 

Our simulator allows us to introduce more complicated strategies: add stop loss, profit take, and (in the premium version - not available in this API) even level-driven dynamic strategies.

So let's compute a 'real life' Sharpe Ratio by setting up the mean of the process as the profit target: 

In [4]:

# defining the api-endpoint 
def MarkovSimulator(Mu, Theta, Sigma, currentValue, ProcessType = "MeanReverting", NumberSimulations = 100000, 
                    NumberForwardPeriods = 182, UpperStop = 999999.0, LowerStop = -999999.0,
                    PointValue = 1.0, InitialNumberContracts = 1, DynamicStrategy = []): 
  API_ENDPOINT = "http://simfunctionapp20180222.azurewebsites.net/api/SimFunction?code=/7CGIJPawdfzJ0kV6Q8ETOHUpiz8r6be2D7irr2Lc2mJJ3Yu/Uzw6w=="
  
  NumberForwardPeriodsBD = int(NumberForwardPeriods * 252 / 365.25)
  json_api = {
    "CalibrationResult": {
      "Mu": Mu,				
        "Sigma": Sigma,				
        "Lambda": Theta,			
        "ProcessType": ProcessType,			
        "PeriodsPerYear": 365.25		
     },
    "SimulationParameters" : {
      "CurrentValue": currentValue,			
        "NumberSimulations": NumberSimulations,		
        "NumberForwardPeriods": NumberForwardPeriodsBD, 		
        "UpperStop": UpperStop,				
        "LowerStop": LowerStop,			
        "PointValue": PointValue,			
        "InitialNumberContracts": InitialNumberContracts,		
        "ThresholdLossProbability": 10000,	
        "DrawdownPercent": 0.5,			
        "DrawdownProbability": 0,		
        "DynamicStrategy": DynamicStrategy
    }
  }
  # data to be sent to api
  # import dynamic_strategy_10_peter
  # r = requests.post(url = API_ENDPOINT, json = dynamic_strategy_10_peter.json_api)
  r = requests.post(url = API_ENDPOINT, json = json_api)
  r_json = r.json() 
  return r_json


# Simulation:
# Long if currentValue < mu, 
# Short if currentValue > mu: 
long_short = sign(mu - currentValue)
in_number_ct = int( long_short )

profit_level = (mu)
stop_loss = -999999.0

# Let's set up the stop levels:
if (long_short * 1 > 0):
  # we are long, UpperStop is profit target
  UpperStop = profit_level
  LowerStop = stop_loss
else:
  # we are short, LowerStop is the target
  LowerStop = profit_level
  UpperStop = -stop_loss

# Simulate  
r_json = MarkovSimulator(mu, theta, sigma, currentValue,
                         NumberForwardPeriods = NumberFP,                         
                         UpperStop = UpperStop,
                         LowerStop = LowerStop,
                         InitialNumberContracts = in_number_ct)

# Results (in r_json) - the following are the useful ones for the Sharpe
ExpectedMean = r_json['SimStatistics']['PnlStatistics']['Mean']
StdDev = r_json['SimStatistics']['PnlStatistics']['StdDev']
UpperStopProbability = r_json['SimStatistics']['PnlStatistics']['UpperStopProbability']
LowerStopProbability = r_json['SimStatistics']['PnlStatistics']['LowerStopProbability']
MeanMaxDrowdown = r_json['DrawdownStatistics']['MeanMaxDrawdown']
MedianMaxDrowdown = r_json['DrawdownStatistics']['MedianMaxDrawdown']

Sharpe = ExpectedMean / StdDev * sqrt (365.25 / NumberFP)

print ("Sharpe ratio: (annual)",Sharpe)


('Sharpe ratio: (annual)', 2.0975558563695853)


Now the ex-ante Sharpe Ratio is close to 2.10! 

A previously 'mediocre' signal has been 'upgraded' from 'good' to 'very good' - still, practitioners will downgrade the signal to allow for ex-post Sharpe Ratio degradation.

For a case study, read [How I maximise my Sharpe Ratio for a mean reverting strategy](https://www.markov.finance/single-post/2018/01/25/How-I-maximise-my-Sharpe-Ratio-for-a-mean-reverting-strategy). Spoiler alert: if you have a 'take profit at mean' you should compensate with a 'stop loss', which will decrease your ex ante Sharpe Ratio - the idea is to move the profit level somewhere between the curren level and the mean and see where it maximizes the Sharpe.


#### Setting up stop losses

**Important: **In a previous [notebook cell](https://colab.research.google.com/drive/1dEjjPPpJ6FVzgZaqp1EjYENSwE7Q9TU8#scrollTo=dz4mcP_1iRcH&line=35&uniqifier=1) I presented a typical mean reverting series with 2 and 3 standard deviation bands plus the Expected forward path and Expected 2 Standard deviation bands. Please open it and see it (I am avoiding recreating it here to reduce notebook code complexity)

Profit-taking targets are quite simple -- the target is usually a number on the way to the mean (and we can simulate many times).

On the other hand, there is a counter-intuitive meaning to the forward expected paths **given** that we start at a level above the 2 standard deviation band: see the expected forward 2 standard deviation graph, and notice how it goes **higher** that the 3 standard (in this case roughly 90bps). Some traders will put the stop loss somewhat lower than the 3 standard deviation band with the belief that there is less than a 0.18% probability of it ever arriving there - and if it were to happen, then they would think the mean reversion was broken.

By looking at the conditional path **given** we start from a high level, at one moment in time the probability of the trade going above 90bps will only be a 2 standard deviation event (roughly 2.3% - see [the 68-95-99.7 rule](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)) - so still 'unlucky', but **not** a black swan.   
  


## Kelly criterion lense

I explained in [Sizing mean reverting trades using the Kelly criterion](https://www.markov.finance/single-post/2018/02/20/Sizing-mean-reverting-trades-using-the-Kelly-criterion) a method to look at mean reversion from a 'biased coin' perspective (which allow you to use Kelly to weight how much to allocate to a mean reverting trade). Below you can see a method to compute the probabilities:

In [5]:

# set the horizon at the half life,
# upper stop and lower stop simmetrically, at the standard devition /2 to make the band
# 'tight' enough to force stops.
r_json = MarkovSimulator(mu, theta, sigma, currentValue, 
                         NumberForwardPeriods = int(half_life), 
                         UpperStop = currentValue + std_band/2, 
                         LowerStop = currentValue - std_band/2)

# The simulator runs the process, and counts the number of times it has reached the
# upper stop to compute the frequency:
UpperStopProbability = r_json['SimStatistics']['PnlStatistics']['UpperStopProbability']

# and similarly for the lower stop:
LowerStopProbability = r_json['SimStatistics']['PnlStatistics']['LowerStopProbability']

print ('Biased probabilities:', UpperStopProbability, LowerStopProbability)

# pick up the maximum probability - we will 'bet' on it
max_prob = max( [UpperStopProbability, LowerStopProbability])

# Kelly criterion for biased coins (but which the market offers 1:1 odds):
print ('Kelly =', max_prob * 2 - 1)

('Biased probabilities:', 0.29709, 0.69039)
('Kelly =', 0.3807799999999999)


in this example (because the last value is above the mean) the LowerStopProbability has a 69% probability, which corresponds to a 38% Kelly ratio. As it is often said:



>  [[quote](https://books.google.co.uk/books?id=A5pI1fewuAoC&pg=PA243&lpg=PA243&dq=edge+over+the+market+51%25&source=bl&ots=hBmmjWFItU&sig=smewovQrSPp66esV6DR7GXzPh8o&hl=en&sa=X&ved=0ahUKEwjdpLfQk4raAhXPblAKHblcCPYQ6AEIJzAA#v=onepage&q=edge%20over%20the%20market%2051%25&f=false)] Just being right 51% is enough to give you an edge over the market - 70% makes you an all-time legend. But that still means super-legends must be comfortable being wrong 30% of the time.

A practitioner would see the Kelly allocation as "[the red line that should never be crossed](http://www.financial-math.org/blog/2013/10/two-tales-of-the-kelly-formula/)" and would usually allocate a fraction of Kelly ("half the kelly [=] three-quarters of the return with half the volatility" - same [link](https://)).

What happens if there are many simultaneous opportunities ? if they are independent, we can use [Kelly Criterion for multiple simultaneous events](https://www.linkedin.com/pulse/kelly-criterion-multiple-simultaneous-events-gerardo-lemus/) - if the other signals offer better payouts this weight will decrease).
 

## Remarks

Initially, we would disregard the pca trade due to it being arbitrarily below a 2 multiple ratio, and having a so-so 1.5 ex-ante SharpeRatio, but  
* after forward simulations, we found the 'real life' ex-ante sharpe could change the perspective of entering trades (by adding the profit targets and stop losses explicitely), and
* we also have a formula (Kelly) to optimally allocate the risk in the case if the trader want to maximize wealth. 

Dedicated traders can now use this (in sample) triggers and levels and test the (out of sample) performance in an algorithmic strategy, or use it as inputs from a discretionary book.


